### Get Stop Words ###

In [ ]:
from gasp3.dt.txtcls.san import get_stop_words

intbl  = '/home/jasp/todo/geotm/text_cls/tbl_ref/ref_fire_facebook_2017_2018.xlsx'
outtbl = '/home/jasp/todo/geotm/text_cls/tbl_ref/test_fire_face_1718_sw.xlsx'

conParam = {
    'HOST': 'localhost', 'PORT' : '5432', 'PASSWORD' : 'admin',
    'USER' : 'postgres', 'TEMPLATE' : 'postgis_template'
}

get_stop_words(
    conParam, intbl, 'post_id', 'message', outtbl,
    inSheet='test'
)

### Go Machine Learning ###

In [ ]:
trainData = '/home/jasp/todo/geotm/text_cls/tbl_ref/train_fire_face_1718_sw.xlsx'
classData = '/home/jasp/todo/geotm/text_cls/tbl_ref/test_fire_face_1718_sw.xlsx'

trainSheet = 'train'
trainRefCol  = 'e_incendio'
trainDataCol = 'clean_message'
classDataCol = 'clean_message'

lang = None

outCorrWords = '/home/jasp/todo/geotm/text_cls/tbl_res/tst_corr_words.xlsx'
outPredict   = '/home/jasp/todo/geotm/text_cls/tbl_res/predict_svm.xlsx'
outEval      = '/home/jasp/todo/geotm/text_cls/tbl_res/model_eval.xlsx'
outMxt       = '/home/jasp/todo/geotm/text_cls/tbl_res/mxt_svm.xlsx'

In [ ]:
from gasp3.dt.txtcls import correlated_words

correlated_words(
    trainData, trainRefCol, trainDataCol, outCorrWords,
    lang=lang, N=3, refSheet=trainSheet
)

In [ ]:
from gasp3.dt.txtcls import model_selection

model_selection(trainData, trainRefCol, trainDataCol, outEval, lang=lang)

In [ ]:
from gasp3.dt.txtcls import text_prediction

text_prediction(
    trainData, classData, trainRefCol, trainDataCol,
    classDataCol, outPredict, method='LinearSupportVectorMachine')

In [ ]:
from gasp3.dt.txtcls import model_evaluation

model_evaluation(outPredict, 'is_incendio_v2', 'n_class', outMxt)

### SQL Based Methodology ###

In [ ]:
conParam = {
    "HOST" : "localhost", "PORT" : "5432", "PASSWORD" : "admin",
    "USER" : "postgres" , "DATABASE" : "dsn_collect"
}

TBL_SCHEMA = {
    "TNAME"   : "facedata",
    "TEXTCOL" : "lower_message",
    "TIMECOL" : "datahora",
    "SELCOL"  : ['post_id AS fid', 'type AS tipo', 'message AS mess'],
    "TEXTCASE" : (
        "CASE WHEN type = 'link' THEN lower(unaccent(description)) "
        "ELSE lower(unaccent(message)) END"
    )
}

START = '2017-06-01 00:00:00'
END   = '2018-10-31 23:59:59'

WORDS   = ['incendi', 'fog']

RESULT = '/home/jasp/todo/geotm/text_cls/tbl_res/sql_incendio1718.xlsx'

In [ ]:
from gasp3.dt.txtcls.qbase import get_rows_related_with_event

get_rows_related_with_event(
    conParam, TBL_SCHEMA, WORDS, RESULT,
    startTime=START, endTime=END
)

### Classification Evaluation ###

##### Binary classification evaluation ######

In [ ]:
from gasp3.dt.txtcls.eval import binary_eval

ref_data = '/home/jasp/todo/geotm/text_cls/tbl_ref/test_fire_face_1718_sw.xlsx'
tst_data = '/home/jasp/todo/geotm/text_cls/tbl_res/predict_svm.xlsx'
eval_tbl = '/home/jasp/todo/geotm/text_cls/tbl_res/svm_eval.xlsx'

refIDCol = 'post_id'
refCol   = 'is_incendio_v2'
tstIDCol = 'fid'
tstCol   = 'n_class'

a = binary_eval(
    ref_data, refIDCol, refCol, tst_data, tstIDCol,eval_tbl, tstCol=tstCol
)

In [ ]:
from gasp3.fm import tbl_to_obj
from gasp3.to import obj_to_tbl

ref_df = tbl_to_obj(ref_data, useFirstColAsIndex=None)
tst_df = tbl_to_obj(tst_data, useFirstColAsIndex=None)

In [ ]:
ref_df.head(5)

In [ ]:
tst_df.head(5)

In [ ]:
df = ref_df.merge(tst_df, how='left', left_on='post_id', right_on='fid')

In [ ]:
df.head(5)